In [876]:
# this block imports all necessary libraries and a necessary thing is for this code to work, run ganache 
# and replace the address and publickeys as are required.


from eth_utils import address
from web3 import Web3
import os
import solcx
from solcx import compile_standard, install_solc
from dotenv import load_dotenv
import json
from hashlib import sha256

In [877]:
# opening the SCR contract file

with open("./SMR.sol", "r") as file:
    ratingFile = file.read()


In [878]:
#compiling smart contract of certifier(SCR)
install_solc("0.8.2")
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SMR.sol": {"content": ratingFile}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.2",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [879]:
compiled_sol

{'contracts': {'SMR.sol': {'SMR': {'abi': [{'inputs': [{'internalType': 'bytes',
        'name': 'h',
        'type': 'bytes'}],
      'name': 'getHash',
      'outputs': [{'internalType': 'bytes', 'name': '', 'type': 'bytes'}],
      'stateMutability': 'pure',
      'type': 'function'},
     {'inputs': [{'internalType': 'bytes', 'name': 'hash', 'type': 'bytes'}],
      'name': 'getRating',
      'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
      'stateMutability': 'view',
      'type': 'function'},
     {'inputs': [{'internalType': 'bytes',
        'name': 'uniqStrng',
        'type': 'bytes'},
       {'internalType': 'uint256', 'name': 'R', 'type': 'uint256'}],
      'name': 'rateMovie',
      'outputs': [],
      'stateMutability': 'nonpayable',
      'type': 'function'},
     {'inputs': [{'internalType': 'bytes', 'name': 'hash1', 'type': 'bytes'},
       {'internalType': 'bytes', 'name': 'hash2', 'type': 'bytes'}],
      'name': 'submitHashes',
      'ou

In [880]:
#plain SMR is the contract name itself


# get bytecode
bytecode = compiled_sol["contracts"]["SMR.sol"]["SMR"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(
    compiled_sol["contracts"]["SMR.sol"]["SMR"]["metadata"]
)["output"]["abi"]

In [881]:
abi

[{'inputs': [{'internalType': 'bytes', 'name': 'h', 'type': 'bytes'}],
  'name': 'getHash',
  'outputs': [{'internalType': 'bytes', 'name': '', 'type': 'bytes'}],
  'stateMutability': 'pure',
  'type': 'function'},
 {'inputs': [{'internalType': 'bytes', 'name': 'hash', 'type': 'bytes'}],
  'name': 'getRating',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'bytes', 'name': 'uniqStrng', 'type': 'bytes'},
   {'internalType': 'uint256', 'name': 'R', 'type': 'uint256'}],
  'name': 'rateMovie',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [{'internalType': 'bytes', 'name': 'hash1', 'type': 'bytes'},
   {'internalType': 'bytes', 'name': 'hash2', 'type': 'bytes'}],
  'name': 'submitHashes',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'}]

In [882]:
chainID = 1337
chain_id =chainID
httpServer = "HTTP://127.0.0.1:7545"
w3 = Web3(Web3.HTTPProvider(httpServer))

In [883]:
# set up connection for the rating architecture

my_address = '0x9aEA0a629bE7cb6327C36Dd7352AC05bCF396786'
private_key = '0x6998fefb52154033306ee67bd83ccdc32417a1b3c0dd91d19921c6e2d4c8ec54'

In [884]:
def initializeSMR():
        # initialize contract

    SMR = w3.eth.contract(abi=abi, bytecode=bytecode)
    nonce = w3.eth.get_transaction_count(my_address)
    # set up transaction from constructor which executes when firstly we are trasacting the smart contract

    transaction = SMR.constructor().build_transaction(
        {"chainId": chain_id, "from": my_address, "nonce": nonce}
    )
    
    
    # Second requirement sign the transaction
    signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_key)

    # Third requirement is to get the transation hash value

    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

    #  after geeting hash get the recipetpt of the transaction
    
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    return tx_receipt


In [885]:
# Function to store value on SMR

def submitHashes(tx_receipt, hash1, hash2):
    # calling functions in contract block
    # to work with a contract, you need abi and address
    #  this line call the the contract by passing the abi and adress of the transactoin

    registration_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

    #  whenever we want to update something we trsact that value so gas is used for 
    # that purpose we agin perform the transaction

    call_fun = registration_sol.functions.submitHashes(hash1, hash2).build_transaction(
        {"chainId": chain_id, "from": my_address, "nonce": w3.eth.get_transaction_count(my_address)}
    )
    # again sign and same procedure

    sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
    tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
    tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)
    

In [886]:
# Function to store value on SMR

def rateMovie(tx_receipt, unqStr, rating):
    # calling functions in contract block
    # to work with a contract, you need abi and address
    #  this line call the the contract by passing the abi and adress of the transactoin

    registration_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

    #  whenever we want to update something we trsact that value so gas is used for 
    # that purpose we agin perform the transaction

    call_fun = registration_sol.functions.rateMovie(unqStr, rating).build_transaction(
        {"chainId": chain_id, "from": my_address, "nonce": w3.eth.get_transaction_count(my_address)}
    )
    # again sign and same procedure

    sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
    tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
    tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

In [887]:
def getRating(tx_receipt, unqstr):
    #certifier etrieves the submitted encrypted UID by the contract adress 
    registration_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)
    ret = registration_sol.functions.getRating(unqstr).call() 
    print(ret)
    return ret 


In [888]:
def hashConvert(input):
    hash = sha256(input.encode('utf-8')).digest()
    # hash = hash.encode('utf-8')
    print(hash)
    return hash

In [889]:
rec = initializeSMR()
submitHashes(rec, hashConvert('a'), hashConvert('u'))
rateMovie(rec,'u'.encode(), 5)
ret = getRating(rec, hashConvert('u'))
print(ret)


b'\xca\x97\x81\x12\xca\x1b\xbd\xca\xfa\xc21\xb3\x9a#\xdcM\xa7\x86\xef\xf8\x14|Nr\xb9\x80w\x85\xaf\xeeH\xbb'
b'\x0b\xfe\x93^p\xc3!\xc7\xca:\xfcu\xce\r\x0c\xa2\xf9\x8bT"\xe0\x08\xbb1\xc0\x0cm\x7f\x1f\x1c\n\xd6'
b'\x0b\xfe\x93^p\xc3!\xc7\xca:\xfcu\xce\r\x0c\xa2\xf9\x8bT"\xe0\x08\xbb1\xc0\x0cm\x7f\x1f\x1c\n\xd6'
5
5
